## Test de Una muestra (comparamos una muestra con un valor determinado)

In [ ]:
import pandas as pd
#Leemos el dataset

data = pd.read_feather('late_shipments.feather')

data.head()

In [ ]:
#obtenemos el peso medio (weight_kilograms)

meanWeight = data['weight_kilograms'].mean()

print(meanWeight)

1947.682


Cuando cree la empresa hice un estudio de negocio para ver la viabilidad económica de mi empresa de envios dando por hecho que el peso medio de mis pedidos a transportar seria de 2T. ¿Me equivoqué al hacer esa previsión?

In [ ]:
! pip install pingouin

In [ ]:
df[]

In [ ]:
# Realizamos un test de hipotesis
# Ho: meanWeigth = 2T
# H1: meanWeigth != 2T

import pingouin as pg

test_result = pg.ttest(data['weight_kilograms'],2200, alternative='two-sided')

print(test_result)

               T  dof alternative     p-val           CI95%   cohen-d   BF10  \
T-test -2.552994  999        less  0.005414  [-inf, 2110.4]  0.080733  1.817   

           power  
T-test  0.817641  


# Comparación de dos muestras

Creemos que el peso del paquete puede influir en si llega tarde o no. Puede tener sentido que los màs pesados llegen tarde

In [ ]:
weightLate = data[data['late']=='Yes']['weight_kilograms']
weightNoLate = data[data['late']=='No']['weight_kilograms']

print(weightLate.mean())
print(weightNoLate.mean())

2715.6721311475408
1897.7912673056444


In [ ]:
#Hago mi test de hipotesis
# Ho: PesoLate = PesoNoLate X
# H1: PesoLate > PesoNoLate (OK)

test_result = pg.ttest(weightLate,weightNoLate, alternative='greater')

print(test_result)

               T        dof alternative     p-val          CI95%   cohen-d  \
T-test  2.393666  72.527359     greater  0.009635  [248.59, inf]  0.262077   

        BF10     power  
T-test  4.29  0.632045  


## TEST ANOVA (cuando hay varias categorias a valorar)

Creemos que hay diferencias entre el peso de los paquetes en función del transporte. Voy a analizarlo:

In [ ]:
print(data.groupby('shipment_mode')['weight_kilograms'].mean())

shipment_mode
Air            1521.778146
Air Charter    2823.500000
Ocean          6272.840909
Name: weight_kilograms, dtype: float64


In [ ]:
# Ho: El peso entre los diferentes transportes es el mismo X
# H1: El peso entre los diferentes transportes es diferente OK

anova_result = pg.anova(data=data,dv='weight_kilograms',between='shipment_mode')

print(anova_result)


          Source  ddof1  ddof2           F         p-unc       np2
0  shipment_mode      2    997  113.920752  2.765091e-45  0.186017


In [ ]:
# Hacemos ttest entre todas las combinaciones de categorias posibles
pairwise_result = pg.pairwise_tests(data=data,dv='weight_kilograms',between='shipment_mode')

print(pairwise_result)

        Contrast            A            B  Paired  Parametric         T  \
0  shipment_mode          Air  Air Charter   False        True -1.683731   
1  shipment_mode          Air        Ocean   False        True -9.013237   
2  shipment_mode  Air Charter        Ocean   False        True -3.716732   

         dof alternative         p-unc       BF10    hedges  
0   5.121589   two-sided  1.516616e-01      1.048 -0.512097  
1  91.642098   two-sided  2.805384e-14  3.965e+15 -1.679632  
2  10.507752   two-sided  3.673659e-03     62.766 -0.717724  
